# 3D Molecular Visualization with NovoMD

This notebook demonstrates how to visualize molecules using the 3D coordinates returned by NovoMD.

## Requirements

```bash
pip install py3Dmol matplotlib plotly
```

In [ ]:
import requests
import json
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# Configuration
BASE_URL = "http://localhost:8010"
API_KEY = "your-api-key"
headers = {"Content-Type": "application/json", "X-API-Key": API_KEY}

def get_molecule_data(smiles):
    """Fetch molecule data from NovoMD API"""
    response = requests.post(
        f"{BASE_URL}/smiles-to-omd",
        headers=headers,
        json={"smiles": smiles, "force_field": "AMBER", "add_hydrogens": True}
    )
    return response.json()

## 1. Fetch Molecular Data

In [ ]:
# Get caffeine molecule
smiles = "CN1C=NC2=C1C(=O)N(C(=O)N2C)C"  # Caffeine
result = get_molecule_data(smiles)

if result['success']:
    metadata = result['metadata']
    coords_x = metadata['coords_x']
    coords_y = metadata['coords_y']
    coords_z = metadata['coords_z']
    atom_types = metadata['atom_types']
    bonds = metadata['bonds']
    
    print(f"Molecule: Caffeine")
    print(f"Atoms: {len(atom_types)}")
    print(f"Bonds: {len(bonds)}")
    print(f"Atom types: {set(atom_types)}")

## 2. Simple 3D Plot with Matplotlib

In [ ]:
# Color mapping for atoms
atom_colors = {
    'C': 'gray',
    'N': 'blue',
    'O': 'red',
    'H': 'white',
    'S': 'yellow',
    'P': 'orange',
    'F': 'green',
    'Cl': 'green',
    'Br': 'brown'
}

atom_sizes = {
    'C': 100,
    'N': 100,
    'O': 100,
    'H': 30,
    'S': 120,
    'P': 120
}

fig = plt.figure(figsize=(12, 10))
ax = fig.add_subplot(111, projection='3d')

# Plot atoms
for i, (x, y, z, atom) in enumerate(zip(coords_x, coords_y, coords_z, atom_types)):
    color = atom_colors.get(atom, 'purple')
    size = atom_sizes.get(atom, 80)
    ax.scatter(x, y, z, c=color, s=size, edgecolors='black', linewidth=0.5, alpha=0.9)

# Plot bonds
for bond in bonds:
    i, j = bond
    ax.plot([coords_x[i], coords_x[j]], 
            [coords_y[i], coords_y[j]], 
            [coords_z[i], coords_z[j]], 'k-', linewidth=1, alpha=0.6)

ax.set_xlabel('X (Å)')
ax.set_ylabel('Y (Å)')
ax.set_zlabel('Z (Å)')
ax.set_title('Caffeine - 3D Structure')

# Create legend
from matplotlib.lines import Line2D
legend_elements = [Line2D([0], [0], marker='o', color='w', markerfacecolor=color, 
                          markersize=10, label=atom) 
                   for atom, color in atom_colors.items() if atom in set(atom_types)]
ax.legend(handles=legend_elements, loc='upper left')

plt.tight_layout()
plt.show()

## 3. Interactive 3D Visualization with Plotly

In [ ]:
import plotly.graph_objects as go

# Create traces for atoms
atom_traces = []
for atom_type in set(atom_types):
    indices = [i for i, a in enumerate(atom_types) if a == atom_type]
    x = [coords_x[i] for i in indices]
    y = [coords_y[i] for i in indices]
    z = [coords_z[i] for i in indices]
    
    color = atom_colors.get(atom_type, 'purple')
    size = 15 if atom_type != 'H' else 8
    
    trace = go.Scatter3d(
        x=x, y=y, z=z,
        mode='markers',
        name=atom_type,
        marker=dict(size=size, color=color, line=dict(width=1, color='black'))
    )
    atom_traces.append(trace)

# Create traces for bonds
bond_x, bond_y, bond_z = [], [], []
for bond in bonds:
    i, j = bond
    bond_x.extend([coords_x[i], coords_x[j], None])
    bond_y.extend([coords_y[i], coords_y[j], None])
    bond_z.extend([coords_z[i], coords_z[j], None])

bond_trace = go.Scatter3d(
    x=bond_x, y=bond_y, z=bond_z,
    mode='lines',
    name='Bonds',
    line=dict(color='gray', width=3)
)

# Create figure
fig = go.Figure(data=[bond_trace] + atom_traces)
fig.update_layout(
    title='Caffeine - Interactive 3D Structure',
    scene=dict(
        xaxis_title='X (Å)',
        yaxis_title='Y (Å)',
        zaxis_title='Z (Å)'
    ),
    width=800,
    height=700
)
fig.show()

## 4. Visualization with py3Dmol (PDB-based)

py3Dmol provides publication-quality molecular visualizations:

In [ ]:
try:
    import py3Dmol
    
    # Get PDB content from API response
    pdb_content = result['pdb_content']
    
    # Create viewer
    viewer = py3Dmol.view(width=600, height=500)
    viewer.addModel(pdb_content, 'pdb')
    viewer.setStyle({'stick': {'colorscheme': 'Jmol'}})
    viewer.addStyle({'sphere': {'radius': 0.3, 'colorscheme': 'Jmol'}})
    viewer.zoomTo()
    viewer.show()
except ImportError:
    print("py3Dmol not installed. Install with: pip install py3Dmol")

## 5. Compare Multiple Molecules

In [ ]:
molecules = {
    "Benzene": "c1ccccc1",
    "Naphthalene": "c1ccc2ccccc2c1",
    "Anthracene": "c1ccc2cc3ccccc3cc2c1"
}

fig, axes = plt.subplots(1, 3, figsize=(15, 5), subplot_kw={'projection': '3d'})

for ax, (name, smiles) in zip(axes, molecules.items()):
    result = get_molecule_data(smiles)
    
    if result['success']:
        m = result['metadata']
        
        # Plot atoms
        for x, y, z, atom in zip(m['coords_x'], m['coords_y'], m['coords_z'], m['atom_types']):
            color = atom_colors.get(atom, 'purple')
            size = atom_sizes.get(atom, 80)
            ax.scatter(x, y, z, c=color, s=size, edgecolors='black', linewidth=0.5)
        
        # Plot bonds
        for bond in m['bonds']:
            i, j = bond
            ax.plot([m['coords_x'][i], m['coords_x'][j]], 
                    [m['coords_y'][i], m['coords_y'][j]], 
                    [m['coords_z'][i], m['coords_z'][j]], 'k-', linewidth=1, alpha=0.6)
        
        ax.set_title(f"{name}\n({m['num_heavy_atoms']} heavy atoms)")
        ax.set_xlabel('X')
        ax.set_ylabel('Y')
        ax.set_zlabel('Z')

plt.tight_layout()
plt.show()

## 6. Save Structures

Export the PDB file for use in other visualization tools:

In [ ]:
# Save PDB file
result = get_molecule_data("CN1C=NC2=C1C(=O)N(C(=O)N2C)C")  # Caffeine

if result['success']:
    with open('caffeine.pdb', 'w') as f:
        f.write(result['pdb_content'])
    print("Saved caffeine.pdb")
    
    with open('caffeine.omd', 'w') as f:
        f.write(result['omd_content'])
    print("Saved caffeine.omd")